In [ ]:
import pandas as pd

df1 = pd.read_csv('000007N5_20241217-094027.csv', encoding='cp1252')
df2 = pd.read_csv('000003PS_20241217-093537.csv', encoding='cp1252')

df1 = df1.drop(['typ av konsekvens','2023'], axis=1).loc[df1['2021']!='..']

df1['2021'] = df1['2021'].astype(int)

df1 = df1.groupby('redovisningsgrupp').sum()

df1.rename(index={'Stockholms län': 'Stockholm', 'Norra mellansverige':'Norra Mellansverige', 'Östra mellansverige':'Östra Mellansverige'}, inplace=True)

df2.drop(['typ av IT-tjänst'], axis=1, inplace=True)

df3 = df1.merge(df2, 'left', right_on='redovisningsgrupp', left_on='redovisningsgrupp')

df3.rename(columns={'redovisningsgrupp':'Region', '2021_x':'Incidenter 2021', '2021_y':'Utgifter för IT tjänster 2021'}, inplace=True)

In [11]:
import sqlite3
help(sqlite3.to_sql)

AttributeError: module 'sqlite3' has no attribute 'to_sql'

In [5]:
df3

,Region,Incidenter 2021,Utgifter för IT tjänster 2021
0,Mellersta Norrland,18,730
1,Norra Mellansverige,24,1297
2,Småland med öarna,33,7977
3,Stockholm,30,36930
4,Sydsverige,30,7105
5,Västsverige,30,15656
6,Östra Mellansverige,21,6395
7,Övre Norrland,21,832


In [ ]:
print(df1)

In [88]:
#df3 = df1.merge(df2, 'left', right_on='redovisningsgrupp', left_on='redovisningsgrupp')

In [ ]:
#df2 = df2.loc[df2['2021']!='..']
df1

In [ ]:
#df2 = df2['2021'].astype(int)

In [ ]:
df2["2021"] = df2["2021"].astype(int)

In [ ]:
df2.groupby("redovisningsgrupp")["2021"].agg(sum)